# ONE-HOT ENCODING

In [1]:
import pandas as pd
import numpy as np

**pd.get_dummies()** es útil para un desarrollo simple, si por ejemplo se está llevando a cabo un análisis exploratorio de datos. Este método transforma el DataFrame directamente.
También responde muy bien ante variables nulas (deja todos los valores como 0), funcionalidad que *OneHotEncoder* de *sci-kit learn* no hace.

In [2]:

df = pd.DataFrame({'A':['a1','a2','a3',np.nan],
                   'B':['b1','b2','b3','b3']})

print(pd.get_dummies(df))

    A_a1   A_a2   A_a3   B_b1   B_b2   B_b3
0   True  False  False   True  False  False
1  False   True  False  False   True  False
2  False  False   True  False  False   True
3  False  False  False  False  False   True


En el caso de *OneHotEncoder* es posible sustituir las clases nulas pero se debe tener cuidado con la *multicolinealidad* (La **multicolinealidad** ocurre cuando las variables independientes (predictores) en un modelo de regresión están correlacionadas. Si el grado de correlación entre las variables independientes es elevado, no será posible aislar la relación entre cada variable independiente y la variable dependiente (respuesta)).   


Si se va a realizar un modelo es muy recomendable usar *OneHotEncoder*, ya que este crea una clase que permite hacer *fit* de nuevos datos, conservando las características previas definidas para el modelo, de forma que se vuelve escalable para nuevos datos en ese modelo. No es tan simple como *pd.get_dummies* pero funciona mejor en un pipeline de producción.   

NOTA:   
Es necesario siempre borrar una columna extra en los *dummies* para evitar multicolinealidad: **pd.get_dummies(drop_first=True)** o **(OneHotEncoder(drop=‘first’))**.   
De igual manera, los datos nulos es mejor que sean definidos como otra categoria también para evitar el mismo problema de multicolinealidad.


In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe_class = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_class.fit(df.fillna('nan'))
ohe_class.transform(df.fillna('nan'))

array([[1., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1.]])

EJEMPLO CON VARIOS MÉTODOS DE ONE-HOT ENCODING (OHE)

In [ ]:
# Importamos librerías necesarias
import pandas as pd
import joblib
from sklearn import preprocessing
from category_encoders import one_hot  # pip install category_encoders
# Creating a small DataFrame with fake animal data
animales_df = pd.DataFrame({'animal': ['gato', 'perro', 'pajaro', 'mono', 'elefante', 'gato', 'pajaro']})

In [25]:
animales_df

,animal
0,gato
1,perro
2,pajaro
3,mono
4,elefante
5,gato
6,pajaro


Cómo se observa, de forma intencionada se ha repetido varios tipos de animales para, al pasar el codificador, ver que realmente funciona

In [9]:
# Realizando 'One-Hot encoding' utilizando la función de Pandas "get_dummies"
pandas_dummies = pd.get_dummies(animales_df['animal'])

In [10]:
pandas_dummies

,elefante,gato,mono,pajaro,perro
0,False,True,False,False,False
1,False,False,False,False,True
2,False,False,False,True,False
3,False,False,True,False,False
4,True,False,False,False,False
5,False,True,False,False,False
6,False,False,False,True,False


Usando la libreria OneHotEncoder de Scikit-Learn

In [11]:
# Instanciando el objeto OHE de Scikit-Learn
sklearn_ohe = preprocessing.OneHotEncoder()

In [12]:
# "Convirtiendo" el DataFrame de animales para transformarlo con el OHE 
# de Scikit-Learn a través de la función sklearn_ohe.fit_transform()
sklearn_dummies = sklearn_ohe.fit_transform(animales_df)


In [13]:
# Usamos el output de "dummies" y las categorías ya transformadas para 
# obtener un aspecto más claro de nuestro dataframe
sklearn_dummies_df = pd.DataFrame(data = sklearn_dummies.toarray(), 
                                  columns = sklearn_ohe.categories_)

In [14]:
# Se realiza el volcado del objeto "transformador" a un fichero externo 
# usando el formato *.pickle
joblib.dump(sklearn_ohe, 'sklearn_ohe.pkl')

['sklearn_ohe.pkl']

Otra posibilidad, como ya se ha visto, es usando la librería de Python denominada **Category_Encoders**.    

Para su instalación:   
```python
pip install category_encoders # desde el terminal de sistema

conda install -c conda-forge category_encoders # desde el terminal de anaconda
```


In [15]:
# Instanciamos el objeto OHE de la librería "Category Encoders"
ce_ohe = one_hot.OneHotEncoder(use_cat_names = True)

In [16]:
# "Convirtiendo" el DataFrame de animales para transformarlo con el OHE 
# de "Category Encoders"
ce_dummies = ce_ohe.fit_transform(animales_df)

In [17]:
# Se realiza el volcado del objeto "transformador" a un fichero externo 
# usando el formato *.pickle
joblib.dump(ce_ohe, 'ce_ohe.pkl')

['ce_ohe.pkl']

Para probar que estos transformadores están haciendo lo que se espera que hagan, se reimportan desde sus archivos .pickle exportados y se realiza la codificación en una nueva versión del DataFrame de animales. Es bastante similar al original, pero con los valores reordenados.

In [18]:
# Creacion de un nuevo DataFrame de animales
new_animales_df = pd.DataFrame({'animal': ['gato', 'gato', 'perro', 'mono', 'elefante', 'gato', 'pajaro', 'perro']})


In [19]:
# Importamos los ficheros pickle que se han exportado previamente
imported_sklearn_ohe = joblib.load('sklearn_ohe.pkl')
imported_ce_ohe = joblib.load('ce_ohe.pkl')


In [20]:
# Ejecución de los datos transformados sobre el nuevo dataframe de animales
# 1º - Usando OHE de Scikit-Learn
loaded_sklearn_dummies = imported_sklearn_ohe.transform(new_animales_df)
loaded_sklearn_dummies_df = pd.DataFrame(data = loaded_sklearn_dummies.toarray(), 
                                         columns = imported_sklearn_ohe.categories_)


In [21]:
# 2º - Usando OHE de Category Encoders
loaded_ce_dummies = imported_ce_ohe.transform(new_animales_df)

In [22]:
loaded_ce_dummies

,animal_gato,animal_perro,animal_pajaro,animal_mono,animal_elefante
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,0,1,0
4,0,0,0,0,1
5,1,0,0,0,0
6,0,0,1,0,0
7,0,1,0,0,0


Cada una de las transformaciones de salida comparten las mismas características que se vieron por primera vez al realizar la transformación en el conjunto de datos original.